<a href="https://colab.research.google.com/github/multi-proj/API/blob/master/K_means%EB%B6%84%EC%84%9D_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Reference

- WikiDocs

### File Preprocessing

In [0]:
# xlsx to csv

import xlrd
import csv

def csv_from_excel():
    wb = xlrd.open_workbook('test.xlsx')
    sh = wb.sheet_by_name('test')
    your_csv_file = open('test.csv', 'w')
    wr = csv.writer(your_csv_file, quoting=csv.QUOTE_ALL)

    for rownum in range(sh.nrows):
        wr.writerow(sh.row_values(rownum))

    your_csv_file.close()

# runs the csv_from_excel function:
csv_from_excel()

In [0]:
import pandas as pd
import numpy as np

from konlpy.tag import *
from gensim.models import Word2Vec
import gensim.models

hannanum = Hannanum()
kkma = Kkma()
komoran = Komoran()
okt = Okt()

In [0]:
from konlpy.corpus import kobill
from gensim import corpora

### 데이터 불러오기

In [21]:
test_1 = pd.read_csv('test.csv')
# csvreader = csv.reader(x.replace('\0', '') for x in test_1)
test_1.fillna('')
test_1.head()

,명칭,소속,왕(인물),왕(시기),유형,분류,행정 구,연도,설명1,설명2
0,지광국사현묘탑,NaN,NaN,문종(고려),승탑,국보,종로구,1075년,1962년 12월 20일 국보 제101호로 지정되었다. 전체 높이는 6.1m이다. ...,지광국사탑은 고려시대 조정이 최고의 승려에게 내리는 법계인 ‘국사’(國師)를 받은 ...
1,홍화문,창경궁,NaN,"성종(건립), 광해군(재건)","궁궐, 궁문",보물,종로구,1484년,"보물 제384호. 우진각지붕집. 정면 3칸, 측면 2칸의 다포계 중층 문루. 148...","정면 3칸, 측면 2칸의 중층(2층) 우진각지붕으로 동쪽으로 향한 점이 특징인 대문..."
2,창녕위궁재사,NaN,NaN,순조,"건축, 재사",등록문화재,강북구,1832년,조선 제23대 순조의 둘째 딸 복온공주(福溫公主:1818~1832)와 부마 창녕위(...,"1832년에 복온공주가 세상을 떠난 뒤 무덤이 주변에 자리하였고, 이어서 김병주의 ..."
3,후원,창덕궁,NaN,태종,"유적, 정원",사적,종로구,1405년,면적 10만 3000여 평. 사적 제122호로 지정된 창덕궁 안에 있는 조선시대 정...,창덕궁 후원(昌德宮後苑) 또는 비원(祕苑)은 창덕궁 북쪽에 창경궁과 붙어 있는 한국...
4,창빈안씨묘역,NaN,"숙종, 선조",명종,묘,시도유형문화재,동작구,1683년,1983년에 서울시유형문화재 제54호로 지정되었다. 조선 제11대 중종의 후궁(後宮...,창빈 안씨 묘역(昌嬪 安氏 墓域)은 조선 제11대 임금 중종(1488∼1544)의 ...


### Data Preprocessing

- Concatenate several columns in one

In [0]:
test_1['명칭_유형'] = list(i[-2:] for i in test_1['명칭'])
test_1['명칭_유형'].head()
test_type = test_1['명칭_유형']

use_col = ['명칭', '명칭_유형', '연도', '행정 구', '분류', '유형', '소속']


- K-means용 프레임데이터

In [23]:
test_1['설명1'] = test_1['설명1'].apply(hannanum.nouns)
test_sulmyung = test_1['설명1']
# hannanum.nouns(list(i for i in test_1['명칭'])) 

test_sulmyung.head()

0    [1962년, 12월, 20일, 국보, 제101호, 지정, 전체, 6., 1m, 강...
1    [보물, 제384호, 우진각지붕집, 정면, 3칸, 측면, 2칸, 다포계, 중층, 문...
2    [조선, 제23대, 순조, 둘째, 복온공주(福溫公主:1818~1832), 부마, 창...
3    [면적, 10, 3000, 평, 사적, 제122호, 지정, 창덕궁, 안, 조선시대,...
4    [1983년, 서울시유형문화재, 제54호, 지정, 조선, 제11대, 중종, 후궁(後...
Name: 설명1, dtype: object

In [24]:
type(test_sulmyung[0])

list

In [25]:
type(test_1['명칭_유형'][0])

str

In [0]:
for j in range(0,len(use_col)):
  
  for i in range(len(test_sulmyung)):
    test_sulmyung[i].append(test_1[use_col[j]][i])

In [27]:
test_sulmyung[0]

['1962년',
 '12월',
 '20일',
 '국보',
 '제101호',
 '지정',
 '전체',
 '6.',
 '1m',
 '강원',
 '원주시',
 '부론면(富論面)',
 '법천리(法泉里)',
 '국권침탈',
 '직후',
 '일본',
 '반출',
 '반환',
 '경복궁',
 '지광국사현묘탑',
 '묘탑',
 '1075년',
 '종로구',
 '국보',
 '승탑',
 nan]

In [56]:
# test_sulmyung=test_sulmyung.str.join(" ")
# test_sulmyung[0]

# I have more args that might be None / or not exist like and that breaks
# str(' ').join(test_sulmyung)
'''for i in range(len(test_sulmyung)):
  filter(None, test_sulmyung[i])'''

for i in range(len(test_sulmyung)):
  test_sulmyung[i] = list(filter(None, test_sulmyung[i])) # fastest

  
pd.isnull(test_sulmyung[0])

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False,  True])

In [29]:
test_sulmyung.shape
# pd.concat([test_sulmyung, test_add], axis=1)    

(121,)

- KNN용 프레임데이터

In [30]:
test_1['설명2'] = test_1['설명2'].apply(hannanum.nouns)
test_sulmyung_knn = test_1['설명2']
test_sulmyung_knn.head()

0    [지광국사탑, 고려시대, 조정, 최고, 승려, 법계, ‘국사’(國師), 지광국사, ...
1    [정면, 3칸, 측면, 2칸, 중, 2층, 우진각지붕, 동쪽, 점, 특징, 대문, ...
2    [1832년, 복온공주, 세상, 뒤, 무덤, 주변, 자리하였고,, 이, 김병주, 무...
3    [창덕궁, 후원(昌德宮後苑), 비원(祕苑), 창덕궁, 북쪽, 창경궁, 한국, 최대,...
4    [창빈, 안씨, 묘역(昌嬪, 安氏, 墓域, 조선, 제11대, 임금, 중종(1488∼...
Name: 설명2, dtype: object

In [31]:
type(test_sulmyung_knn[0])

list

In [32]:
test_sulmyung_knn=test_sulmyung.str.join(" ")
" ".join(test_sulmyung_knn[0])

TypeError: ignored

In [0]:
test_sulmyung_knn.shape

### Training

In [0]:
from gensim.models import Word2Vec

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier  

In [0]:
'''# 사용할 데이터 Word2Vec포멧으로 embedding시키기

embedding_model2 = Word2Vec(test_sulmyung, min_count = 1)
embedding_model2'''

In [0]:
type(test_sulmyung)

#### K-means

In [0]:
X_train = test_sulmyung
X_train.head()

In [0]:
# training


tfidf = TfidfVectorizer()
dataset1 = tfidf.fit_transform(X_train)

from scipy.sparse import csr_matrix
dataset1_mat = csr_matrix(dataset1, dtype=np.int8).toarray()
dataset1_mat
# k-means에는 바로 적용가능하지만 다른 적용을 위해서는 numpy형태로 바꿔줘야해서 array배열로 바꿔주는 작업이 필요하다.(.toarray())

from sklearn.decomposition import PCA
PCA1 = PCA(n_components = 5)
PCA2 = PCA1.fit_transform(dataset1_mat)

In [0]:
cluster_number = 15
km1=KMeans(n_clusters=cluster_number)
out1=km1.fit_predict(dataset1)

####  KNN

In [0]:
from sklearn.neighbors import KNeighborsClassifier

In [0]:
X2_train = test_sulmyung_knn
X2_train.head()

In [0]:
# training


tfidf = TfidfVectorizer()
dataset2 = tfidf.fit_transform(X2_train)
dataset2.shape

'''
# from scipy.sparse import csr_matrix
dataset2_mat = csr_matrix(dataset2, dtype=np.int8).toarray()
dataset2_mat

# from sklearn.decomposition import PCA
PCA2_1 = PCA(n_components = 5)
PCA2_2 = PCA2_1.transform(dataset2_mat)

neighbors_number = 5
knn1 = KNeighborsClassifier(n_neighbors=neighbors_number)
knn.fit(dataset2)
'''

out2 = knn1.predict(dataset2)


Reference Code
```
tf1 = TfidfVectorizer()
tf2 = tf1.fit_transform(data)
type(tf2)

knn1 = KNeighborsClassifier(n_neighbors=5)
knn1.fit(tf2, out1)

from scipy.sparse import scr_matrix
tf3 = csr_matrix(tf2, dtype=np.int8).toarray()

from sklearn.cluster import KMeans
km1 = KMeans(n_clsuter=3)
out1 = km1.fit_predict(tf3)
txt2 = txt1[out==k]

tf4 = tf1.transform(data) # tf2와 달리 fit과정을 거치지 않는다
tf4.shape

out2 = knn1.predict(tf4)
```

### 예측값 정확도 확인하기

In [0]:
'''
cluster_number = ?
'''
out1

In [0]:
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# x = np.random.normal(size=100)
sns.distplot(out1, bins=cluster_number); # , fit=stats.gamma

In [0]:
test_1['명칭'][out1==3]

### Plot